In [1]:
# HÜCRE 1
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# HÜCRE 3
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
print("✅ Ortam değişkenleri ayarlandı")

✅ Ortam değişkenleri ayarlandı


In [3]:
# HÜCRE 4
!pip install -q "protobuf<6.0,>=5.26.1"
!pip install -q "torch>=2.4.0" --index-url https://download.pytorch.org/whl/cu121
!pip install -q --upgrade \
    "transformers>=4.51.3" \
    "datasets>=3.3.0" \
    "accelerate==1.4.0" \
    "peft==0.14.0" \
    "trl==0.12.1" \
    "bitsandbytes==0.45.3"
!pip install -q sentencepiece tqdm
print("✅ Tüm kütüphaneler başarıyla kuruldu!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.1/342.1 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 46.6 MB/s eta 0:00:00
✅ Tüm kütüphaneler başarıyla kuruldu!


In [4]:
# HÜCRE 5
from huggingface_hub import notebook_login
notebook_login()

In [5]:
# HÜCRE 6
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    BitsAndBytesConfig
)
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
import gc

print(f"✅ PyTorch: {torch.__version__}")
print(f"✅ CUDA: {torch.cuda.is_available()}")
print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
print(f"\n📊 GPU Bellek:")
print(f"   Toplam: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
print(f"   Kullanılabilir: {(torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated(0)) / 1024**3:.2f} GB")

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


✅ PyTorch: 2.9.0+cu126
✅ CUDA: True
✅ GPU: NVIDIA L4

📊 GPU Bellek:
   Toplam: 22.16 GB
   Kullanılabilir: 22.16 GB


In [6]:
# HÜCRE 7
DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/data.jsonl"

import os
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"Dataset bulunamadı: {DATA_PATH}")

print("📂 Dataset yükleniyor...")
dataset = load_dataset("json", data_files={"train": DATA_PATH})

print("\n📊 Dataset Bilgileri:")
print(dataset)
print(f"\n✅ Toplam örnek sayısı: {len(dataset['train'])}")

first_example = dataset['train'][0]
required_keys = ['question', 'answer']
missing_keys = [k for k in required_keys if k not in first_example]

if missing_keys:
    raise ValueError(f"Dataset'te eksik alanlar: {missing_keys}")

print("\n📝 İlk 3 örnek:")
for i in range(min(3, len(dataset['train']))):
    example = dataset['train'][i]
    q_preview = example['question'][:80] + "..." if len(example['question']) > 80 else example['question']
    a_preview = example['answer'][:80] + "..." if len(example['answer']) > 80 else example['answer']
    print(f"\n{i+1}. Örnek:")
    print(f"   Soru: {q_preview}")
    print(f"   Cevap: {a_preview}")

print("\n✅ Dataset başarıyla yüklendi ve doğrulandı!")

📂 Dataset yükleniyor...


Generating train split: 0 examples [00:00, ? examples/s]


📊 Dataset Bilgileri:
DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 391
    })
})

✅ Toplam örnek sayısı: 391

📝 İlk 3 örnek:

1. Örnek:
   Soru: Jeotermal Kaynaklar ve Doğal Mineralli Sular Kanunu'nun kanun numarası nedir?
   Cevap: 5686

2. Örnek:
   Soru: Jeotermal Kaynaklar ve Doğal Mineralli Sular Kanunu ne zaman kabul edilmiştir?
   Cevap: 3/6/2007

3. Örnek:
   Soru: Jeotermal Kaynaklar ve Doğal Mineralli Sular Kanunu hangi tarihte Resmî Gazete'd...
   Cevap: 13/6/2007

✅ Dataset başarıyla yüklendi ve doğrulandı!


In [7]:
# HÜCRE 8
MODEL_ID = "google/gemma-3-4b-it"

print(f"🔄 Tokenizer hazırlanıyor: {MODEL_ID}")

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID,
    trust_remote_code=True
)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print(f"\n✅ Tokenizer yüklendi")
print(f"   Vocab size: {len(tokenizer)}")
print(f"   EOS token: {tokenizer.eos_token} (ID: {tokenizer.eos_token_id})")
print(f"   PAD token: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")

🔄 Tokenizer hazırlanıyor: google/gemma-3-4b-it


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]


✅ Tokenizer yüklendi
   Vocab size: 262145
   EOS token: <eos> (ID: 1)
   PAD token: <eos> (ID: 1)


In [8]:
# HÜCRE 9
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_storage=torch.bfloat16,
    llm_int8_enable_fp32_cpu_offload=True
)

print("✅ Quantization config hazır")

✅ Quantization config hazır


In [9]:
# HÜCRE 10
print("🔄 Gemma 3 4B modeli yükleniyor...")
print("⏰ Bu işlem 5-10 dakika sürebilir...")

gc.collect()
torch.cuda.empty_cache()

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map={"": 0},
    trust_remote_code=True,
    attn_implementation="eager",
    torch_dtype=torch.bfloat16
)

model.config.use_cache = False

print("\n✅ Model başarıyla yüklendi!")
print(f"📊 Model parametreleri: {model.num_parameters():,}")

memory_footprint = model.get_memory_footprint() / 1024**3
print(f"💾 Model bellek kullanımı: {memory_footprint:.2f} GB")

gc.collect()
torch.cuda.empty_cache()

🔄 Gemma 3 4B modeli yükleniyor...
⏰ Bu işlem 5-10 dakika sürebilir...


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]


✅ Model başarıyla yüklendi!
📊 Model parametreleri: 4,300,079,472
💾 Model bellek kullanımı: 2.95 GB


In [10]:
# HÜCRE 11
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

print("✅ LoRA konfigürasyonu hazırlandı")
print(f"\n📊 LoRA Parametreleri:")
print(f"   Rank (r): {lora_config.r}")
print(f"   Alpha: {lora_config.lora_alpha}")

✅ LoRA konfigürasyonu hazırlandı

📊 LoRA Parametreleri:
   Rank (r): 8
   Alpha: 16


In [11]:
# HÜCRE 12
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
print("\n✅ LoRA başarıyla uygulandı!")

trainable params: 5,949,440 || all params: 4,306,028,912 || trainable%: 0.1382

✅ LoRA başarıyla uygulandı!


In [12]:
# HÜCRE 13
def formatting_prompts_func(examples):
    questions = examples["question"]
    answers = examples["answer"]
    texts = []

    for question, answer in zip(questions, answers):
        messages = [
            {"role": "user", "content": question},
            {"role": "assistant", "content": answer}
        ]

        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False
        )

        texts.append(text)

    return {"text": texts}

print("🔄 Dataset formatlanıyor...")
formatted_dataset = dataset["train"].map(
    formatting_prompts_func,
    batched=True,
    remove_columns=dataset["train"].column_names,
    desc="Formatting dataset"
)

print(f"✅ Dataset formatlandı: {len(formatted_dataset)} örnek")
print("\n📝 Formatlanmış örnek:")
print(formatted_dataset[0]["text"][:500] + "...")

🔄 Dataset formatlanıyor...


Formatting dataset:   0%|          | 0/391 [00:00<?, ? examples/s]

✅ Dataset formatlandı: 391 örnek

📝 Formatlanmış örnek:
<bos><start_of_turn>user
Jeotermal Kaynaklar ve Doğal Mineralli Sular Kanunu'nun kanun numarası nedir?<end_of_turn>
<start_of_turn>model
5686<end_of_turn>
...


In [13]:
# HÜCRE 14
output_dir = "./gemma3-4b-lora-finetuned"

dataset_size = len(formatted_dataset)
per_device_batch = 1
grad_accum = 16
epochs = 3

effective_batch_size = per_device_batch * grad_accum
steps_per_epoch = dataset_size // effective_batch_size
total_steps = steps_per_epoch * epochs

print(f"📊 Training Konfigürasyonu:")
print(f"   Dataset boyutu: {dataset_size}")
print(f"   Per-device batch: {per_device_batch}")
print(f"   Gradient accumulation: {grad_accum}")
print(f"   Effective batch size: {effective_batch_size}")
print(f"   Total steps: {total_steps}")

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_batch,
    gradient_accumulation_steps=grad_accum,
    num_train_epochs=epochs,
    learning_rate=2e-4,
    weight_decay=0.01,
    warmup_steps=5,
    optim="paged_adamw_8bit",
    bf16=True,
    gradient_checkpointing=True,
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=1,
    dataloader_num_workers=0,
    remove_unused_columns=True,
    report_to="none",
)

print("\n✅ Training arguments hazırlandı!")

📊 Training Konfigürasyonu:
   Dataset boyutu: 391
   Per-device batch: 1
   Gradient accumulation: 16
   Effective batch size: 16
   Total steps: 72

✅ Training arguments hazırlandı!


In [14]:
# HÜCRE 15
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=formatted_dataset,
    dataset_text_field="text",
    max_seq_length=256,
    tokenizer=tokenizer,
    packing=False,
)

print("✅ Trainer oluşturuldu!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/391 [00:00<?, ? examples/s]

✅ Trainer oluşturuldu!


In [15]:
# HÜCRE 16
import time
from datetime import datetime

print("="*70)
print("🚀 EĞİTİM BAŞLIYOR!")
print("="*70)
print(f"⏰ Başlangıç: {datetime.now().strftime('%H:%M:%S')}")
print("="*70)

start_time = time.time()

train_result = trainer.train()

end_time = time.time()
duration = end_time - start_time

print("\n" + "="*70)
print("✅ EĞİTİM TAMAMLANDI!")
print("="*70)
print(f"⏰ Bitiş: {datetime.now().strftime('%H:%M:%S')}")
print(f"⌛ Süre: {duration/60:.2f} dakika")
print(f"📊 Loss: {train_result.training_loss:.4f}")
print("="*70)

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 1, 'bos_token_id': 2, 'pad_token_id': 1}.


🚀 EĞİTİM BAŞLIYOR!
⏰ Başlangıç: 23:29:45


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,4.969200
20,3.189000
30,2.374500
40,2.021300
50,1.897000
60,1.769800
70,1.642400



✅ EĞİTİM TAMAMLANDI!
⏰ Bitiş: 23:42:33
⌛ Süre: 12.79 dakika
📊 Loss: 2.4957


In [16]:
# HÜCRE 17
final_model_path = "./gemma3-4b-lora-final"

trainer.save_model(final_model_path)

print(f"✅ Model kaydedildi: {final_model_path}")

!mkdir -p "/content/drive/MyDrive/Colab Notebooks/fine_tuned_models/"
!cp -r {final_model_path} "/content/drive/MyDrive/Colab Notebooks/fine_tuned_models/"
print("✅ Drive'a yedeklendi")

✅ Model kaydedildi: ./gemma3-4b-lora-final
✅ Drive'a yedeklendi
